In [1]:
##Calculating Piotroski's F score
import numpy as np
import pandas as pd
from tabula import read_pdf as rp
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time

In [2]:
## setting up browser ##returns driver
def browser_setup():
    url="https://www.capitalmarket.com/Company-Information/Getquote"
    chrome_options = Options();
    chrome_options.add_argument("--headless")
    chrome_options.binary_location = 'C:/Users/Harshit Mishra/AppData/Local/Google/Chrome SxS/Application/chrome.exe'    
    driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"),options=chrome_options)  
    driver.get(url)
    driver.implicitly_wait(10)
    return driver

In [3]:
#open page for bs of a firm and return urls for financial statements
def extract_link(company_name,driver):
    search_bar = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtSearchScheme"]')
    search_bar.clear()
    search_bar.send_keys(company_name);
    drpdwn= driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_pnlGetQuote"]/div/div/div/div[2]/div[3]/div/div')
    drpdwn.click()
    bs= driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_pnlGetQuote"]/div/div/div/div[2]/div[3]/div/ul/li[8]')
    bs.click();
    go = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_pnlGetQuote"]/div/div/div/div[3]/div[2]/div')
    go.click();
    time.sleep(5);
    link_bs=driver.current_url;
    link_pnl= link_bs.replace("Balance-sheet", "Profit-and-Loss");
    link_quartres=link_bs.replace("Balance-sheet","Quarterly-Results");
    return [link_bs,link_pnl,link_quartres];

https://www.capitalmarket.com/Company-Information/Financials/Balance-sheet/Maruti-Suzuki-India-Ltd/5496
https://www.capitalmarket.com/Company-Information/Financials/Profit-and-Loss/Maruti-Suzuki-India-Ltd/5496
https://www.capitalmarket.com/Company-Information/Financials/Quarterly-Results/Maruti-Suzuki-India-Ltd/5496


['https://www.capitalmarket.com/Company-Information/Financials/Balance-sheet/Maruti-Suzuki-India-Ltd/5496',
 'https://www.capitalmarket.com/Company-Information/Financials/Profit-and-Loss/Maruti-Suzuki-India-Ltd/5496',
 'https://www.capitalmarket.com/Company-Information/Financials/Quarterly-Results/Maruti-Suzuki-India-Ltd/5496']

In [4]:
def open_page(driver,link): ##opens the financial statement page
    driver.get(link);
    driver.implicitly_wait(10);
    return driver;

In [5]:
def extract_bs(driver): ##returns the balance sheet for firm as a dataframe
    years=[];
    values=pd.DataFrame();
    for head in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_BalanceSheet"]/table/thead'):
            for y in head.find_elements_by_xpath('./tr/th'):
                years.append(y.get_attribute('textContent'));
    for body in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_BalanceSheet"]/table/tbody'):
            for r in body.find_elements_by_xpath('./tr'):
                val=pd.DataFrame();
                for data in r.find_elements_by_xpath('./td'):
                    val=val.append([data.get_attribute('textContent')],ignore_index=True);
                values=values.append(val.T,ignore_index=True)

    values.columns=years;
    return values;

In [6]:
def extract_pnl(driver): ##returns the balance sheet for firm as a dataframe
    years=[];
    values=pd.DataFrame();
    for head in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_ProfitLoss"]/table/thead'):
            for y in head.find_elements_by_xpath('./tr/th'):
                years.append(y.get_attribute('textContent'));
    for body in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_ProfitLoss"]/table/tbody'):
            for r in body.find_elements_by_xpath('./tr'):
                val=pd.DataFrame();
                for data in r.find_elements_by_xpath('./td'):
                    val=val.append([data.get_attribute('textContent')],ignore_index=True);
                values=values.append(val.T,ignore_index=True)

    values.columns=years;
    return values;

In [7]:
def extract_quartres(driver): ##returns the balance sheet for firm as a dataframe
    years=[];
    values=pd.DataFrame();
    for head in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_QtrlyResult"]/table/thead'):
            for y in head.find_elements_by_xpath('./tr/th'):
                years.append(y.get_attribute('textContent'));
    for body in driver.find_elements_by_xpath('//*[@id="ContentPlaceHolder1_td_QtrlyResult"]/table/tbody'):
            for r in body.find_elements_by_xpath('./tr'):
                val=pd.DataFrame();
                for data in r.find_elements_by_xpath('./td'):
                    val=val.append([data.get_attribute('textContent')],ignore_index=True);
                values=values.append(val.T,ignore_index=True)

    values.columns=years;
    return values;

In [13]:
def get_statements(name):
    company_name=name;
    driver=browser_setup();
    links=extract_link(company_name,driver);
    driver=open_page(driver,links[0]);
    bs=extract_bs(driver);
    driver=open_page(driver,links[1]);
    pnl=extract_pnl(driver);
    driver=open_page(driver,links[2]);
    quart_res=extract_quartres(driver);
    return [bs,pnl,quart_res];

In [14]:
##Defining Petroski's variables
#Profitability
sheets=get_statements("Maruti Suzuki India Ltd")

https://www.capitalmarket.com/Company-Information/Financials/Profit-and-Loss/Maruti-Suzuki-India-Ltd/5496


In [19]:
bs=sheets[0]
pnl=sheets[1]
quart_res=sheets[2]

In [64]:
6,593.20

,Particulars,Mar 2019 ( ).Cr,Mar 2018 ( ).Cr,Mar 2017 ( ).Cr,Mar 2016 ( ).Cr,Mar 2015 ( ).Cr,Mar 2014 ( ).Cr
0,SOURCES OF FUNDS :,,,,,,
1,Share Capital,151.00,151.00,151.00,151.00,151.00,151.00
2,Reserves Total,45990.50,41606.30,36280.10,29733.20,23553.20,20827.00
3,Equity Share Warrants,0.00,0.00,0.00,0.00,0.00,0.00
4,Equity Application Money,0.00,0.00,0.00,0.00,0.00,0.00
5,Total Shareholders Funds,46141.50,41757.30,36431.10,29884.20,23704.20,20978.00
6,Secured Loans,0.00,0.00,0.00,0.00,0.00,1363.50
7,Unsecured Loans,149.60,110.80,483.60,230.90,515.60,460.40
8,Total Debt,149.60,110.80,483.60,230.90,515.60,1823.90
9,Total Liabilities,46291.10,41868.10,36914.70,30115.10,24219.80,22801.90


In [58]:
##Defining Pietroskei ratios
TA_0=float(bs[bs["Particulars"]=="Total Assets"].iloc[0,1]) ##Total Assets
TA_1=float(bs[bs["Particulars"]=="Total Assets"].iloc[0,2]) ##Previous Year's Total Assets
NI_0=float(pnl[pnl["Particulars"]=="Reported Net Profit"].iloc[0,1]) ##Net income
NI_1=float(bs[pnl["Particulars"]=="Reported Net Profit"].iloc[0,2]) ##Previous Year's Total Assets
#Operating Cash flow
#ocf = operating profit - other income - Stock Adjustments
ocf = float(bs[pnl["Particulars"]=="Operating Profit"])-float(bs[pnl["Particulars"]=="Other Income"])-float(bs[pnl["Particulars"]=="Stock Adjustments"])


C:\Users\Harshit Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\Harshit Mishra\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


TypeError: float() argument must be a string or a number, not 'DataFrame'

In [19]:
##Profitability Ratios 4
f_roa=0; #return on assets #
#roa=Net income before extraordinary items/total assets at the beginning of the year 
roa = NI_0/TA_1; 
if(roa >0):f_roa=1;

f_ocf=0; 
# ocf = net cash flow from operating activities / total assets at the beginning of the year
ocf = 
if(CF/TA>0):ocf=1;

acc=0 ##accurals
if(roa-ocf<0):acc=1;

del_roa=0; #del_roa
if(roa[i-1]-roa[i]>0):del_roa=1;

#Capital Structure ratios 3
##change in leverage 
lev1=ltd/avg_lta; 
lev0=ltd0/avg_lta0;
del_lev=lev1-lev0;
if(del_lev<0):
    f_dellev=1;
else:
    f_dellev=0;

#change in liquidity
f_del_liquid=0; 
curr_rat1=curr_asset1/cur_liab1; ## Current ratio
curr_rat0=curr_asset0/cur_liab0;
del_liquid=curr_rat1-curr_rat0;
if(del_liquid>0): f_del_liquid=1;

eq_offer = 0; #if company has raised equity money in last one year put it one

##Efficiency Measures
f_del_margin=0; #del_margin
gmr1=g_margin/total_sales; ##gross margin ratio
gmr0=g_margin0/total_sales0;
del_margin= gmr1-gmr0;
if(del_margin>0):f_delmargin=1;

#del_turnover
f_del_turnover=0;
atr1=total_sales/total_asset_at_beg;
atr0=total_sales0/total_asset_at_beg0;
del_turnover=atr1-atr0;
if(del_turnover>0):f_del_turnover=1;


In [ ]:
#Petroski's F_score
f_profitability=f_roa+f_ocf+f_accurals+f_del_roa;
f_capital=f_dellev+f_delliq+f_eq_offer;
f_efficiency=f_del_turnover+f_del_margin;

f_score= f_profitability+f_capital+f_efficiency